In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import time

In [14]:
DATA_DIR = './data/'
train_labels = pd.read_csv(DATA_DIR + 'train_labels.csv', index_col='building_id')
submission_format = pd.read_csv(DATA_DIR + 'submission_format.csv', index_col='building_id')
cat_train = pd.read_csv('train_pseudo_catboost_prediction.csv')
cat_test = pd.read_csv('test_pseudo_catboost_prediction.csv')
lgbm_train = pd.read_csv('train_pseudo_lightgbm_prediction.csv')
lgbm_test = pd.read_csv('test_pseudo_lightgbm_prediction.csv')
xgboost_train = pd.read_csv('train_pseudo_xgboost_prediction.csv')
xgboost_test = pd.read_csv('test_pseudo_xgboost_prediction.csv')

In [15]:
train_labels = train_labels.reset_index(drop=True)
cat_train.drop('Unnamed: 0', axis=1, inplace=True)
cat_test.drop('building_id', axis=1, inplace=True)
lgbm_train.drop('Unnamed: 0', axis=1, inplace=True)
lgbm_test.drop('building_id', axis=1, inplace=True)
xgboost_train.drop('Unnamed: 0', axis=1, inplace=True)
xgboost_test.drop('building_id', axis=1, inplace=True)

In [16]:
stack_x_train = pd.concat((cat_train, lgbm_train, xgboost_train), axis = 1)
stack_x_test = pd.concat((cat_test, lgbm_test, xgboost_test), axis = 1)
stack_x_train.shape

(260601, 9)

In [17]:
N_SPLITS = 5
strat_kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
scores = np.empty(N_SPLITS)
ensemble_prediction = np.zeros((stack_x_test.shape[0],3))
for idx, (train_idx, test_idx) in enumerate(strat_kf.split(stack_x_train, train_labels)):
    print("=" * 12 + f"Training fold {idx}" + 12 * "=")
    start = time.time()

    X_train, X_val = stack_x_train.iloc[train_idx], stack_x_train.iloc[test_idx]
    y_train, y_val = train_labels.iloc[train_idx], train_labels.iloc[test_idx]    

    lr = LogisticRegression(random_state = 42)
    lr.fit(X_train, y_train)
    
    preds = lr.predict(X_val)
    score = f1_score(y_val, preds, average='micro')
    scores[idx] = score

    y_pred = lr.predict_proba(stack_x_test)
    ensemble_prediction += y_pred

    runtime = time.time() - start
    print(f"Fold {idx+1} finished with score: {score:.5f} in {runtime:.2f} seconds.\n")

ensemble_prediction /= N_SPLITS
print('CV mean: {:.4f}, CV std: {:.4f}'.format(np.mean(scores), np.std(scores)))

============Training fold 0============


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold 1 finished with score: 0.75189 in 3.64 seconds.

============Training fold 1============


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Fold 2 finished with score: 0.75236 in 4.17 seconds.

============Training fold 2============
Fold 3 finished with score: 0.75501 in 4.05 seconds.

============Training fold 3============


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold 4 finished with score: 0.75543 in 3.87 seconds.

============Training fold 4============


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fold 5 finished with score: 0.75203 in 4.01 seconds.

CV mean: 0.7533, CV std: 0.0015


In [18]:
#ensemble_prediction = np.array(ensemble_prediction)
prediction_labels = np.argmax(ensemble_prediction, axis=1)+1
submission = pd.DataFrame(data=prediction_labels,
                             columns=submission_format.columns,
                             index=submission_format.index)
submission.to_csv('ensemble_submission.csv')
submission.value_counts()

damage_grade
2               55862
3               24644
1                6362
dtype: int64